In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
# url = 'https://sustainabledevelopment.un.org/partnership/?p=23155'
url = 'https://sustainabledevelopment.un.org/partnership/?p=26498'
raw_data = requests.get(url).text

soup = BeautifulSoup(raw_data, 'html.parser')


home_right = soup.find('div', attrs={'class':'homeRight'})

home_right_raw_lst = home_right.getText().split('\n')
home_right_raw_lst = list(filter(None, home_right_raw_lst))

In [3]:
# Title
def get_title():
    return soup.find(id = 'headline').get_text()

# Goals
def get_goals():
    goals_raw = soup.find(id='targets')
    goals_lst = [goal.get_text() for goal in goals_raw.findAll('strong')]
    return ','.join(goals_lst)

# Partners
def get_partners():
    home_right = soup.find('div', attrs={'class':'homeRight'})

    home_right_raw_lst = home_right.getText().split('\n')
    home_right_raw_lst = list(filter(None, home_right_raw_lst))
    partner_index = home_right_raw_lst.index('Partners') + 1
    partners = home_right_raw_lst[partner_index] if home_right_raw_lst[partner_index] != 'Countries' else ''
    return 'Partners : ' + partners 


# Description
def get_description():
    des_raw = soup.find(id='intro').find('div', attrs={'class':'wrap'})
    des_lst = []

    temp=re.sub( r'<p><\/p>', '', str(des_raw))
    temp=temp.replace('</div>','')


    for des in temp.split('<div id="subHeadline">'):
        temp = re.sub(r'[\r\n]+', ' : ', des.strip())
        temp = re.sub(r'[\x91\x92]', '\'', temp)
        des_lst.append(temp)

    # remove stuff in the begining
    try:
        des_lst.remove('<div class="wrap" style="margin-bottom:10px;">')
    except:
        pass
    
    return ' || '.join(des_lst)

def get_resources():
    resources_raw = soup.find(id='resources')
    resources_lst = []

    for resource in resources_raw.findAll('div', recursive = False):
        temp = resource.get_text()

        if temp != '':
            temp = re.sub(r'\n+', ' : ', temp.strip())
#             temp = re.sub(r'[\x91\x92]', '\'', temp)
            resources_lst.append(temp.strip())
    return ' || '.join(resources_lst)


def get_timeframe():
    time_frame_index  = [i for i, item in enumerate(home_right_raw_lst) if re.search('Time-frame', item)]
    return home_right_raw_lst[time_frame_index[0]] if len(time_frame_index) == 1 else 'Time-frame: '

def get_countries():
    countries_index = home_right_raw_lst.index('Countries') + 1
    contact_index = home_right_raw_lst.index('Contact information')
    countries = home_right_raw_lst[countries_index:contact_index] if countries_index != contact_index else ''
    return 'Countries : ' + ",".join(countries)

In [4]:
functions = [get_title, get_goals, get_partners, get_description, get_resources, get_timeframe, get_countries]

for f in functions:
    print('\n --------- \n')
    print(f.__name__)
    print('\n')
    print(f())


 --------- 

get_title


Mare Nullius? Sea-level Rise and Maritime Sovereignities in the Pacific - An Expanded Anthropology of Climate Change

 --------- 

get_goals


Goal 13,Goal 14,Goal 17

 --------- 

get_partners


Partners : University of Bergen (Norway); The University of the South Pacific (Fiji,  Cook Islands, Kiribati, Marshall Islands, Nauru, Niue, Samoa, Solomon Islands, Tokelau, Tonga, Tuvalu, Vanuatu); University of Hawai'i at Manoa (United States); Australian National University; University of Auckland; Pratt Institute; PSIDS Group of Ambassadors at the United Nations.

 --------- 

get_description


Description/achievement of initiative : This project, funded by the Research Council of Norway,  aims to build new and urgent interdisciplinary analysis and policy recommendations concerning of one of the greatest challenges of our time: how Pacific islanders and their states prepare for, and react to, what may be the demise of their sovereign land and sea territories. In t